This notebook contains my approach to McKinsey ProHack Challange 2020. In this work i have investigate multiple approach to similar problems and try to create a hybrid solution. The thing that can make my solution special can be usage of knnimputer for handling missing values and usage of regressor's parameter tuning via hyperopt library.
I train and test the model on the train data only since i can't measure losses on the test data.

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from math import sqrt
import statsmodels.api as sm
from sklearn.metrics.regression import mean_absolute_error as mae
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, cross_val_score
from hyperopt import hp, fmin, tpe
from hyperopt.pyll import scope
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.regression module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
train=pd.read_csv('train.csv')
train.sample(5)

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GGP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 creatures ages 15-64),"Adolescent birth rate (births per 1,000 female creatures ages 15-19)",Total unemployment rate (female to male ratio),Vulnerable employment (% of total employment),"Unemployment, total (% of labour force)",Employment in agriculture (% of total employment),Labour force participation rate (% ages 15 and older),"Labour force participation rate (% ages 15 and older), female",Employment in services (% of total employment),"Labour force participation rate (% ages 15 and older), male",Employment to population ratio (% ages 15 and older),Jungle area (% of total land area),"Share of employment in nonagriculture, female (% of total employment in nonagriculture)",Youth unemployment rate (female to male ratio),"Unemployment, youth (% ages 15–24)","Mortality rate, female grown up (per 1,000 people)","Mortality rate, male grown up (per 1,000 people)","Infants lacking immunization, red hot disease (% of one-galactic year-olds)","Infants lacking immunization, Combination Vaccine (% of one-galactic year-olds)",Gross galactic product (GGP) per capita,"Gross galactic product (GGP), total","Outer Galaxies direct investment, net inflows (% of GGP)",Exports and imports (% of GGP),Share of seats in senate (% held by female),Natural resource depletion,"Mean years of education, female (galactic years)","Mean years of education, male (galactic years)","Expected years of education, female (galactic years)","Expected years of education, male (galactic years)","Maternal mortality ratio (deaths per 100,000 live births)",Renewable energy consumption (% of total final energy consumption),"Estimated gross galactic income per capita, male","Estimated gross galactic income per capita, female",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GGP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GGP),"Remittances, inflows (% of GGP)",Population with at least some secondary education (% ages 25 and older),Intergalactic inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary under-age population)","Respiratory disease incidence (per 100,000 people)",Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII),y
1834,1000000,Carina Dwarf (E206-G220),0.683066,68.168081,24615.241111,0.691643,12.463787,6.907088,0.558809,0.367607,NaN,102.488207,69.492450,26.14549,1270.75047,51.635952,141.410527,111.576541,16.085961,664.201756,60.0

In [3]:
#encoding categorical columns
le = preprocessing.LabelEncoder()
train['galaxy']=le.fit_transform(train['galaxy'])
train['galactic year']=le.fit_transform(train['galactic year'])


In [4]:
#missing values handled with knnimputer
imp = KNNImputer(weights='distance')
train=pd.DataFrame(imp.fit_transform(train.copy()), columns=train.columns)

In [5]:
# Choosing high correlation  columsn for training
corr = train.corrwith(train['y'])
dictt = corr.to_dict()
Corrlist = []
for key,value in dictt.items():
    if value >= 0.3:
        Corrlist.append(key)
    elif value <= - 0.3:
        Corrlist.append(key)
train_final = train[Corrlist]
train_final=train_final.drop('y', axis=1)

In [6]:
#a great script for parameter tuning, i can't remember the source
scope.define(GradientBoostingRegressor)    

def train_GradientBoostingRegressor(Xdata, Ydata, loss='ls' ,alpha = 0.50, cv = 2, n_steps = 10):
    #split data
    X_train, X_test, y_train, y_test = train_test_split(Xdata, Ydata, test_size=.33, random_state = 42)
    
    # create and objective function
    def objective_function_regression(estimator):
        mae_array = cross_val_score( estimator, X_train, y_train, cv= cv, n_jobs=-1, 
                                    scoring = make_scorer(mae) )
        return mae_array.mean()


    # search space
    n_estimators     = hp.randint('n_estimators',1000) 
    learning_rate    = hp.loguniform('learning_rate',-3,1)
    max_depth        = hp.randint('max_depth', 10)
    max_features     = hp.randint('max_features',X_train.shape[1])
    min_samples_leaf = hp.randint('min_samples_leaf', 10)
    #criterion        = hp.choice('criterion', ['friedman_mse'])
                                       
    # model / estimator to be optimized
    est0 = (0.1, scope.GradientBoostingRegressor( n_estimators  = n_estimators + 1,
                                            learning_rate = learning_rate,
                                            max_depth = max_depth + 1,
                                            max_features = max_features + 1,
                                            min_samples_leaf = min_samples_leaf + 1,
                                            random_state= 101) 
        )

    search_space_regression = hp.pchoice('estimator', [est0])
    
    best = fmin(
    fn= objective_function_regression,
    space= search_space_regression,
    algo = tpe.suggest, # This is the optimization algorithm hyperopt uses, a tree of parzen estimators
    max_evals = n_steps,
    verbose = 0  # The number of iterations
         )
    
    # Allocate optimized parameters and apply to test data set
    Regressor = GradientBoostingRegressor( learning_rate = best['learning_rate'],
                          max_depth = best['max_depth'],
                          max_features = best['max_features'],
                          min_samples_leaf = best['min_samples_leaf'],
                          n_estimators = best['n_estimators'],
                          random_state = 101
                                       )
    # fit
    Regressor.fit(X_train,y_train)
    
    #evaluate
    yhat = Regressor.predict(X_test) ;
    error_pct = np.round( np.median(np.abs(yhat - y_test)), 2)
    #print('--'*20)
    
    print("{} {}".format('The Median Abs. Error (%) for the test set is :', error_pct))
    
    return Regressor, y_test, yhat 

In [7]:
train_GradientBoostingRegressor(train_final, train['y'], loss='ls' ,alpha = 0.50, cv = 2, n_steps = 10)

The Median Abs. Error (%) for the test set is : 0.01


(GradientBoostingRegressor(learning_rate=0.07906541393315811, max_features=42,
                           min_samples_leaf=6, n_estimators=326,
                           random_state=101), 2323    0.065814
 1419    0.062036
 2477    0.043814
 1583    0.084893
 952     0.051737
 2828    0.052332
 1551    0.215541
 657     0.047905
 321     0.059225
 3216    0.053752
 1727    0.125833
 1767    0.048105
 3299    0.199074
 1833    0.049185
 1861    0.059595
 2330    0.045011
 1578    0.061639
 1924    0.176147
 32      0.056974
 2948    0.059456
 1689    0.059064
 2771    0.192255
 2934    0.039425
 2416    0.041944
 2321    0.075737
 14      0.050220
 415     0.046817
 2080    0.042470
 2046    0.044717
 2883    0.058928
 1614    0.099067
 2623    0.047550
 527     0.061207
 1681    0.199428
 315     0.131499
 410     0.052006
 1720    0.065283
 1818    0.050051
 3623    0.282051
 1987    0.164947
 2271    0.100174
 2500    0.084662
 1284    0.143779
 2107    0.133096
 3325    0.053595
 